In [14]:
%matplotlib inline

import numpy as np
import collections
from collections import OrderedDict, Counter, defaultdict
import pandas as pd

import Bio
from Bio import SeqIO

import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show 
import matplotlib.patches as patches    
from matplotlib import colors


import glob

import subprocess
from subprocess import call

import re

import pickle

from Bio.SubsMat import MatrixInfo
import Bio.Data.CodonTable

from scipy import stats


from Bio.SubsMat import MatrixInfo
import Bio.Data.CodonTable

import joypy

# Metadata

In [15]:
path = '/home/alla/make_PhD_great_again/NTE/VERSION_since_May2021/'

# gencode 25
metadata_pc_g25 = pd.read_csv(path+'tmp_res/metadata_pc_g25.txt', sep='\t')

metadata_pc_g25['tr_id1'] = [x.split('.')[0] for x in metadata_pc_g25['tr_id'].tolist()]

In [16]:
# open metadata with scores and metrics
meta = pd.read_csv('/home/alla/make_PhD_great_again/NTE/VERSION_since_May2021/tmp_res/METADATA_CURRENT.txt', sep='\t')

meta[0:2]

,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,...,gene_ext,Start_ext,Stop_ext,global_rank_ext,Start_codon_ext,Proteomics_count_ext,Coverage_value_ext,CDS_ratio,Riboseq_Summary,Nstart_Nend
0,ENST00000624431.1,274,316,42,274,316,14.0,FO538757.2,274,chr1:182667-182708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,274-316
1,ENST00000624735.1,0,236,236,2,236,78.0,FO538757.1,86,chr1:200087-200236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2-236


# Get sequences of extensions 

In [17]:
for el in meta[(meta['Riboseq_Summary'].notna()) & (meta['gene'].isin(['CCDC8', 'PYGO2', 'RGS19', 'STK38', 'SLC25A32', 'MT1F']))][['tr_id', 'gene', 'Riboseq_Summary']].merge(metadata_pc_g25[['tr_id', 'transcript_seq']], on='tr_id', how='inner').to_numpy():
    tr_id = el[0]
    gene = el[1]
    summary = el[2]
    start = int(summary.split('; ')[1].split('-')[0])
    stop = int(summary.split('; ')[1].split('-')[1])
    tr_seq = el[-1]
    
    if gene == 'CCDC8':
        start = 565
    print (gene, tr_id, start, stop+3)
    print ('nonAUG_AUG seq', tr_seq[start-1:stop+2])
    #print ('hairpin d14-17nt')
    #print ('10nt upstream', tr_seq[start-11:start-1])
    print ('100nt downstrem', tr_seq[start-1:start-1+100])
    print ()

PYGO2 ENST00000368457.2 44 176
nonAUG_AUG seq CTGTCGTCGCACTTTGTGGTTGCTGCAGCTCGGGGGCCTGGGCTGCCCCTGACACCCCTTCTGGGCGATGGTGCAGCCCAAGGGCGCCTCCATCCCCCGCCGCTGCCGCTAACCCGGGTCCCCCACTCCATG
100nt downstrem CTGTCGTCGCACTTTGTGGTTGCTGCAGCTCGGGGGCCTGGGCTGCCCCTGACACCCCTTCTGGGCGATGGTGCAGCCCAAGGGCGCCTCCATCCCCCGC

STK38 ENST00000229812.7 221 290
nonAUG_AUG seq CTGGCCGAAGGGGCGTCTGCGGCTCTCCGCCGTCCCCAGCCCTGCCTCTCCCTGGGCTCTGCAGCCATG
100nt downstrem CTGGCCGAAGGGGCGTCTGCGGCTCTCCGCCGTCCCCAGCCCTGCCTCTCCCTGGGCTCTGCAGCCATGGCAATGACAGGCTCAACACCTTGCTCATCCA

SLC25A32 ENST00000297578.8 30 171
nonAUG_AUG seq TTGGGCATAAGAGTCCTCTCGTTGGTCCCGGAGGTGGGGTTGCGCTCACAAGGGGCGACCGTCGCCACGGTGGCGGCCACTGCATCGCGTCCCACCTCCGCGGCCCTGGGCGCCGTGGTGTCGACGGGCCCCGAGCCTATG
100nt downstrem TTGGGCATAAGAGTCCTCTCGTTGGTCCCGGAGGTGGGGTTGCGCTCACAAGGGGCGACCGTCGCCACGGTGGCGGCCACTGCATCGCGTCCCACCTCCG

MT1F ENST00000334350.6 336 369
nonAUG_AUG seq TTGGAAAGTCCAGTCTCTCCTCGGCTTGCAATG
100nt downstrem TTGGAAAGTCCAGTCTCTCCTCGGCTTGCAATGGACCCCAACTGCTCCTGCGCCGCTGGTGTC

In [18]:
# PYGO2 no eligible haipins 
# STK38 no
# 

In [19]:
for el in metadata_pc_g25[metadata_pc_g25['gene'].isin(['GTF3A', 'EIF4G2', 'TRPV6'])][['tr_id', 'gene', 'cds_start_pos', 'transcript_seq']].to_numpy():
    tr_id = el[0]
    gene = el[1]
    cds_start_pos = el[2]
    tr_seq = el[3]
    
    
    print (gene, tr_id)
    print ('50nt downstream of nonAUG', tr_seq[cds_start_pos:cds_start_pos+50])
    #print ('10nt upstream', tr_seq[cds_start_pos-10:cds_start_pos])
    print ('100nt downstrem', tr_seq[cds_start_pos:cds_start_pos+100])
    print ()

TRPV6 ENST00000359396.7
50nt downstream of nonAUG ACGGGACCTCTACAGGGAGACGGTGGGCCGGCCCTTGGGGGGGCTGATGT
100nt downstrem ACGGGACCTCTACAGGGAGACGGTGGGCCGGCCCTTGGGGGGGCTGATGTGGCCCCAAGGCTGAGTCCCGTCAGGGTCTGGCCTCGGCCTCAGGCCCCCA

TRPV6 ENST00000436401.1
50nt downstream of nonAUG ATGACCCCTAAGGACGATATCCGGCTGGTCGGGGAGCTGGTGACTGTCAT
100nt downstrem ATGACCCCTAAGGACGATATCCGGCTGGTCGGGGAGCTGGTGACTGTCATTGGGGCTATCATCATCCTGCTGGTAGAGGTTCCAGACATCTTCAGAATGG

TRPV6 ENST00000431833.1
50nt downstream of nonAUG ATGCCAGGAGCCATGGGGGAAACAGCGCTACACATAGCAGCCCTCTATGA
100nt downstrem ATGCCAGGAGCCATGGGGGAAACAGCGCTACACATAGCAGCCCTCTATGACAACCTGGAGGCCGCCATGGTGCTGATGGAGGCTGCCCCGGAGCTGGTCT

EIF4G2 ENST00000526148.5
50nt downstream of nonAUG GTGGAGAGTGCGATTGCAGAAGGGGGTGCTTCTCGTTTCAGTGCTTCTTC
100nt downstrem GTGGAGAGTGCGATTGCAGAAGGGGGTGCTTCTCGTTTCAGTGCTTCTTCGGGCGGAGGAGGAAGTAGGGGTGCACCTCAGCACTATCCCAAGACTGCTG

EIF4G2 ENST00000525681.5
50nt downstream of nonAUG GTGGAGAGTGCGATTGCAGAAGGGGGTGCTTCTCGTTTCAGTGCTTCTTC
100nt downstrem GTGGA

In [20]:
'CGC	CCC	GCG	CTT	CCC	AGG	AGG	TTC	ATC	TGC	TCC	TTC	CCT	GAC	TGC	AGC	GCC	AAT	TA'.replace('\t', '')

'CGCCCCGCGCTTCCCAGGAGGTTCATCTGCTCCTTCCCTGACTGCAGCGCCAATTA'

# Proteomics/peptidomics 

In [21]:
GTF3A: CUG in mammals, but no N-term-ext in fish/amphibia.
    
Any proteomics data? 

Openprot 

SyntaxError: invalid syntax (<ipython-input-21-da30a0751835>, line 1)

# Scan for m6A 